# 14_Saving Data to Multiple Sheets in a Single Excel File

Sometimes it may be desirable to save data from a Pandas DataFrame into multiple sheets of an Excel file.


### Example:
Suppose you have time series data that you want to split up into weekly reports, and save each week as a separate tab in an Excel workbook.

### Prepare the DataFrame:

In [1]:
import pandas as pd

# Load the example daily time series dataset:
df = pd.read_csv("./data_etc/timeseries_daily.csv")

# Convert the 'Date' column from string to DateTime:
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)

df.head()

,Date,feature_1,feature_2,feature_3,feature_4,categorical_feature,weekday
0,2017-02-01,0,0,37,0,foo,Wednesday
1,2017-02-02,0,0,168,0,foo,Thursday
2,2017-02-03,0,0,157,0,other,Friday
3,2017-02-04,0,0,720,0,other,Saturday
4,2017-02-05,0,0,721,0,bar,Sunday


In [2]:
# Add the week column:
df["Week"] = df["Date"].dt.week
df.head()

,Date,feature_1,feature_2,feature_3,feature_4,categorical_feature,weekday,Week
0,2017-02-01,0,0,37,0,foo,Wednesday,5
1,2017-02-02,0,0,168,0,foo,Thursday,5
2,2017-02-03,0,0,157,0,other,Friday,5
3,2017-02-04,0,0,720,0,other,Saturday,5
4,2017-02-05,0,0,721,0,bar,Sunday,5


In [3]:
# Add a column which counts the number of days in the week:
df["WeekLength"] = df.groupby("Week")["Week"].transform("count")

# Only keep full weeks:
df = df.drop(df[df["WeekLength"] < 7].index)
df.reset_index(inplace=True, drop=True)

# Convert the date back to pretty string format:
df["Date"] = df["Date"].dt.strftime("%d/%m/%Y")

df.head()

,Date,feature_1,feature_2,feature_3,feature_4,categorical_feature,weekday,Week,WeekLength
0,06/02/2017,0,0,136,0,other,Monday,6,7
1,07/02/2017,0,0,151,0,bar,Tuesday,6,7
2,08/02/2017,0,0,127,0,other,Wednesday,6,7
3,09/02/2017,0,0,133,0,bar,Thursday,6,7
4,10/02/2017,0,0,129,0,foo,Friday,6,7


### Write the data to an Excel file:
Now that the DataFrame is ready, we can use a loop to write out the weekly data to tabs in a single spreadsheet, using <code>pd.ExcelWriter()</code> and <code>pd.to_excel()</code>.

In [4]:
# Create the writer to the output Excel file:
writer = pd.ExcelWriter("./data_etc/example_multi_sheet_excel_output.xlsx")

# Iterate through the weeks.
# Caution! Note that this loop wouldn't work as expected if  
# the DataFrame contained more than one year's data; there
# would be multiple weeks with the same week number.
for w in df["Week"].unique():
    
    week = df[df["Week"]==w][["Date","feature_1",
                              "feature_2", "feature_3",
                              "feature_4", "categorical_feature"]]
    
    # Write the week to an Excel sheet:
    week.to_excel(excel_writer=writer, sheet_name="week_{}".format(w), index=False)
    
# Save the file after the loop.
writer.save()